In [1]:
import sys
sys.path.append('..') 
from src.vdb_managers.chroma_manager import ChromaManager

In [2]:
chroma_store = ChromaManager() # 自定义的chroma类

In [6]:
file_path = "files/论文 - GraphRAG.pdf"
# file_path = "files/上奇实习报告.pdf"
auto_extract_metadata = True
metadata = {
    'keywords': "实习", # 必须是str，好像bool等也行，不记得了，反正不能是list
    "from": "上齐实习"
}
collection_name = "RAG"
similarity_metric = "cosine"
dense_weight = 0.5

In [4]:
# 上传PDF文档
chroma_store.upload_pdf_file(file_path = file_path, collection_name = collection_name)
# chroma_store.upload_pdf_file(file_path = file_path, metadata=metadata, collection_name=collection_name)

In [8]:
query = "What are the general methods of deep reinforcement learning?"
k = 3

In [ ]:
results = chroma_store.dense_search(collection_name=collection_name, query=query, k=k)
results

In [ ]:
results = chroma_store.hybrid_search(collection_name=collection_name, query=query, k=k, dense_weight=1)
results

In [12]:
results = chroma_store.search(collection_name=collection_name, query=query, k=k, dense_weight=1)
results

[{'content': 'Recursive abstractive processing for tree-organized retrieval. arXiv preprint arXiv:2401.18059 .\nScott, K. (2024). Behind the Tech. https://www .microsoft .com/en-us/behind-the-tech.\nShao, Z., Gong, Y ., Shen, Y ., Huang, M., Duan, N., and Chen, W. (2023). Enhancing retrieval-\naugmented large language models with iterative retrieval-generation synergy. arXiv preprint\narXiv:2305.15294 .\nSu, D., Xu, Y ., Yu, T., Siddique, F. B., Barezi, E. J., and Fung, P. (2020). Caire-covid: A ques-\ntion answering and query-focused multi-document summarization system for covid-19 scholarly\ninformation management. arXiv preprint arXiv:2005.03975 .\nTang, Y . and Yang, Y . (2024). MultiHop-RAG: Benchmarking retrieval-augmented generation for\nmulti-hop queries. arXiv preprint arXiv:2401.15391 .\nTouvron, H., Martin, L., Stone, K., Albert, P., Almahairi, A., Babaei, Y ., Bashlykov, N., Batra, S.,\nBhargava, P., Bhosale, S., et al. (2023). Llama 2: Open foundation and fine-tuned chat m

In [10]:
formatted_context = chroma_store.get_formatted_context(collection_name=collection_name, query=query, k=k, dense_weight=dense_weight)
formatted_context

'[文档片段 1]\nDataset Example activity framing and generation of global sensemaking questions Podcast transcriptsUser : A tech journalist looking for insights and trends in the tech industry Task: Understanding how tech leaders view the role of policy and regulation Questions : 1. Which episodes deal primarily with tech policy and government regulation? 2. How do guests perceive the impact of privacy laws on technology development? 3. Do any guests discuss the balance between innovation and ethical considerations? 4. What are the suggested changes to current policies mentioned by the guests? 5. Are collaborations between tech companies and governments discussed and how? News articlesUser : Educator incorporating current affairs into curricula Task: Teaching about health and wellness Questions : 1. What current topics in health can be integrated into health education curricula? 2. How do news articles address the concepts of preventive medicine and wellness?\n\n---\n\n[文档片段 2]\nof data sen

### 2. help

In [6]:
import os
from dotenv import load_dotenv # type: ignore

load_dotenv()
api_key = os.getenv('ZETATECHS_API_KEY')
base_url = os.getenv('ZETATECHS_API_BASE')

import chromadb # type: ignore
import chromadb.utils.embedding_functions as embedding_functions # type: ignore

client = chromadb.PersistentClient(path="ChromaVDB")

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=api_key,
    api_base=base_url,
    model_name="text-embedding-3-large"
)

collection = client.get_or_create_collection(name="test", embedding_function=openai_ef)

In [ ]:
help(collection.query)

In [ ]:
help(client.get_or_create_collection)

In [ ]:
help(collection.add)

In [ ]:
help(client.create_collection)